In [ ]:
"""
This notebook 
"""

In [3]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from glob import glob
from scipy import ndimage

from utils import place_in_img

In [ ]:
DEBUG = False

mask_metadata_fps = sorted(glob('../2_live_bees/3_card_mask_matches_metadata/*'))

for mask_metadata_fp in tqdm(mask_metadata_fps[259:]):
    fn = mask_metadata_fp.split('/')[-1]
    out_fp = '../2_live_bees/2_card_masks_in_orig_coords/'+fn.replace('.json','.png')
    #if os.path.exists(out_fp):
    #    continue
    
    card_metadata_fp = '../2_live_bees/1_metadata/' + fn
    with open(mask_metadata_fp, 'r') as f:
        metadata = json.load(f)
    with open(card_metadata_fp, 'r') as f:
        card_metadata = json.load(f)
    metadata.update(card_metadata)
    
    card_angle = metadata['card_angle']
    card_rotated_x_start = metadata['card_rotated_x_start']
    card_rotated_x_end = metadata['card_rotated_x_end']        
    
    date = '_'.join(fn.split('_')[:3])
    bee_id = fn.split('_')[-1].split('.')[0].split('-')[0]
    
    img_fp = glob(f'../WWBEE24_images/*/*/{date}/{bee_id}*')[0]
    mask_fp = '../2_live_bees/2_final_masks/' + fn.replace('.json','.png')
    
    img = cv2.imread(img_fp)
    mask = cv2.imread(mask_fp, cv2.IMREAD_GRAYSCALE)
    
    rotated = ndimage.rotate(img, card_angle, mode='constant', cval=255)
    
    full_mask = np.zeros(rotated.shape[:2]).astype('uint8')
    full_mask[:,card_rotated_x_start:card_rotated_x_end] = mask
    mask_start_x = np.where(full_mask > 0.5)[1].min()
    
    if DEBUG:
        plt.figure()
        plt.imshow(full_mask)
    
    if DEBUG:
        plt.figure()
        plt.imshow(full_mask[:,mask_start_x:])
        
    combined_full_mask = np.zeros(rotated.shape[:2]).astype('uint8')
    combined_full_mask[full_mask > 0.5] = 255
    
    sy, sx = np.array(np.where(combined_full_mask > 0.5)).min(axis=1)
    ey, ex = np.array(np.where(combined_full_mask > 0.5)).max(axis=1)
    
    cv2.imwrite(out_fp, combined_full_mask)
    
    if DEBUG:
        plt.figure()
        plt.imshow(combined_full_mask)
    
    if DEBUG:
        plt.figure()
        plt.imshow(seg)
    
    if DEBUG:
        plt.figure()
        plt.imshow(full_seg)
    
    if DEBUG:
        break

 57%|██████████████████████████████████████████████████████████████████████████████████████████████                                                                        | 529/934 [59:31<45:54,  6.80s/it]

In [14]:
glob(f'../WWBEE24_images/*/*/{date}/{bee_id}*')

['../WWBEE24_images/Round01/Hive03/2024_06_20/h03b44_deformedWing.JPG']

In [11]:
date, bee_id

('2024_06_20', 'h03b44-deformedWing')

In [13]:
bee_id = fn.split('_')[-1].split('.')[0].split('-')[0]